In [620]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [621]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [622]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [623]:
%%capture pwd
!pwd

In [624]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('levelling-up-what-might-it-mean-for-scotland', 300, 500)

# Fig 1

In [625]:
df = pd.read_excel(
    "raw/regionalgrossvalueaddedbalancedperheadandincomecomponents.xlsx",
    skiprows=1,
    sheet_name="Table 2",
)
df = df.dropna(subset=["ITL code"])
df = (
    df[df["ITL"] == "ITL1"]
    .set_index(["ITL", "ITL code", "Region name"])
    .stack()
    .reset_index()
    .drop(["ITL", "ITL code"], axis=1)
)
df.columns = ["region", "year", "gdp_per_head"]
df["year"] = df["year"].replace("2019\n[note 3]", "2019").astype(int)
df = df[df["year"] > 2008]

In [626]:
f = "fig1_gdp_per_head"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,region,year,gdp_per_head
11,North East,2009,17291.0
12,North East,2010,17443.0
13,North East,2011,17648.0
14,North East,2012,18010.0
15,North East,2013,18211.0


In [627]:
base = (
    alt.Chart(f1)
    .encode(
        x=alt.X(
            "year:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                format=".0f",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=4,
                orient="bottom",
                labelAngle=270,
            ),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.region+' | '+datum.year+' | '+datum.gdp_per_head+ ' £/year/head'"
    )
)
line = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.3,
    line={"color": colors["eco-turquiose"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.7),
            alt.GradientStop(color="white", offset=0.1),
        ],
        x1=1,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "gdp_per_head:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridOpacity=0.4,
            title="Gross value added in £/year/head between 2009 and 2019",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            format="s",
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    column=alt.Column(
        "region:N",
        spacing=5,
        header=alt.Header(
            labelAngle=270,
            labelPadding=-38,
            labelAlign="left",
            labelColor=colors["eco-gray"],
            labelFontSize=11,
            title="",
            labelOrient="bottom",
        ),
    ),
)
layer1 = (
    ((line).properties(height=300, width=27))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.Chart(...)

# Fig 2

In [628]:
df = pd.read_excel(
    "raw/regionalgrossvalueaddedbalancedperheadandincomecomponents.xlsx",
    skiprows=1,
    sheet_name="Table 2",
)
df = df.dropna(subset=["ITL code"])
df = df[df["ITL code"].str[:3] == "TLM"]  # filter for scotland
df = (
    df[df["ITL"] == "ITL3"]
    .set_index(["ITL", "ITL code", "Region name"])
    .stack()
    .reset_index()
    .drop(["ITL", "ITL code"], axis=1)
)
df.columns = ["id", "year", "gdp_per_head"]
df["year"] = df["year"].replace("2019\n[note 3]", "2019").astype(int)

# topojson
# https://raw.githubusercontent.com/ONSvisual/topojson_boundaries/master/geogNUTS2018GB.json
scot = json.loads(open("visualisation/fig2/geogNUTS2018GB.json", "r").read())
scot["objects"]["nuts3"]["geometries"] = [
    i
    for i in scot["objects"]["nuts3"]["geometries"]
    if i["properties"]["AREACD"][:3] == "UKM"
]  # filter for scotland
open("visualisation/fig2/geogNUTS2018GB_scot.json", "w").write(json.dumps(scot))
regions = [i["properties"]["AREANM"] for i in scot["objects"]["nuts3"]["geometries"]]

In [629]:
df["id"] = (
    df["id"]
    .replace("City of Edinburgh", "Edinburgh, City of")
    .replace(
        "Caithness and Sutherland and Ross and Cromarty",
        "Caithness & Sutherland and Ross & Cromarty",
    )
    .replace(
        "Inverness and Nairn and Moray, Badenoch and Strathspey",
        "Inverness & Nairn and Moray, Badenoch & Strathspey",
    )
    .replace(
        "Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute",
        "Lochaber, Skye & Lochalsh, Arran & Cumbrae and Argyll & Bute",
    )
    .replace("Na h-Eileanan Siar", "Na h-Eileanan Siar (Western Isles)")
    .replace("Perth and Kinross and Stirling", "Perth & Kinross and Stirling")
    .replace(
        "East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond",
        "East Dunbartonshire, West Dunbartonshire and Helensburgh & Lomond",
    )
    .replace("Dumfries and Galloway", "Dumfries & Galloway")
)
for i in df["id"].unique():
    if not i in regions:
        print(i)

In [630]:
f2 = "fig2_gdp_per_head_map"
open("visualisation/fig2/" + f2 + ".json", "w").write(
    json.dumps(list(df.T.to_dict().values()))
)

39695

In [631]:
readme = (
    "### "
    + f2
    + "\n[Interactive map](https://economicsobservatory.github.io/ECOvisualisations/articles/"
    + uid
    + "/visualisation/fig2/)\n\n"
)
open("README.md", "a").write(readme)

177

# Fig 3

In [632]:
df = pd.read_csv("raw/simd2020_withinds.csv")
df = df[df.columns[:9]]
df.columns = [
    "datazone",
    "zone",
    "area",
    "population",
    "working_age_population",
    "rank",
    "percentile",
    "vigintile",
    "decile",
]
df["year"] = 2020
df = df.dropna(subset=["zone"])

In [633]:
df2 = pd.read_csv("raw/simd2016_withinds.csv")
df2 = df2[df2.columns[:9]]
df2.columns = [
    "datazone",
    "zone",
    "area",
    "population",
    "working_age_population",
    "rank",
    "percentile",
    "vigintile",
    "decile",
]
df2["year"] = 2016
df2["area"] = df2["area"].str.replace("_", " ")
df2["zone"] = df2["zone"].str.replace("_", " ")
df2 = df2.dropna(subset=["zone"])

In [634]:
df = pd.concat([df, df2]).reset_index()
gla = df[df["area"] == "Glasgow City"]
edi = df[df["area"] == "City of Edinburgh"]

In [635]:
f3a = "fig3a_simd_gla"
open("visualisation/fig3/" + f3a + ".json", "w").write(
    json.dumps(list(gla.T.to_dict().values()))
)
f3b = "fig3b_simd_edi"
open("visualisation/fig3/" + f3b + ".json", "w").write(
    json.dumps(list(edi.T.to_dict().values()))
)

288690

In [636]:
scot = json.loads(open("visualisation/fig3/sc_dz_11_micro.json", "r").read())
scot["objects"]["sc_dz_11"]["geometries"] = [
    i
    for i in scot["objects"]["sc_dz_11"]["geometries"]
    if i["properties"]["DataZone"] in gla["datazone"].unique()
]  # filter for glasgow
open("visualisation/fig3/sc_dz_11_micro_gla.json", "w").write(json.dumps(scot))

4678618

In [637]:
scot = json.loads(open("visualisation/fig3/sc_dz_11_micro.json", "r").read())
scot["objects"]["sc_dz_11"]["geometries"] = [
    i
    for i in scot["objects"]["sc_dz_11"]["geometries"]
    if i["properties"]["DataZone"] in edi["datazone"].unique()
]  # filter for glasgow
open("visualisation/fig3/sc_dz_11_micro_edi.json", "w").write(json.dumps(scot))

4631052

In [638]:
readme = (
    "### "
    + f3a
    + "\n[Interactive map](https://economicsobservatory.github.io/ECOvisualisations/articles/"
    + uid
    + "/visualisation/fig3/a.html)\n\n"
)
open("README.md", "a").write(readme)
readme = (
    "### "
    + f3b
    + "\n[Interactive map](https://economicsobservatory.github.io/ECOvisualisations/articles/"
    + uid
    + "/visualisation/fig3/b.html)\n\n"
)
open("README.md", "a").write(readme)

176

# Fig 4

In [639]:
df = pd.read_excel(
    "raw/regionalgrossdisposablehouseholdincomeallitlregions.xls",
    skiprows=1,
    sheet_name="Table 2",
)
df = df.dropna(subset=["ITL code"])
df = df[df["ITL code"].str[:3].isin(["TLM", "UK"])]
df = (
    df[df["ITL"].isin(["ITL2", "UK"])]
    .set_index(["ITL", "ITL code", "Region name"])
    .stack()
    .reset_index()
    .drop(["ITL code"], axis=1)
)
df.columns = ["itl", "region", "year", "gdhi_per_head"]
df["year"] = df["year"].replace("2019\n[note 1]", "2019").astype(int)

In [640]:
f = "fig4_gdhi_per_head"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,itl,region,year,gdhi_per_head
0,UK,United Kingdom,1997,10788.0
1,UK,United Kingdom,1998,11019.0
2,UK,United Kingdom,1999,11528.0
3,UK,United Kingdom,2000,12250.0
4,UK,United Kingdom,2001,12654.0


In [641]:
base = (
    alt.Chart(f4)
    .encode(
        x=alt.X(
            "year:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                format=".0f",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=4,
                orient="bottom",
                labelAngle=270,
            ),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.region+' | '+datum.year+' | '+datum.gdhi_per_head+ ' £/year/head'"
    )
)
line = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.3,
        line={"color": colors["eco-turquiose"]},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color=colors["eco-turquiose"], offset=0.7),
                alt.GradientStop(color="white", offset=0.1),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(
        y=alt.Y(
            "gdhi_per_head:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                gridOpacity=0.4,
                title="Gross disposable household income in £/year/head between 1997 and 2019",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                format="s",
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        opacity=alt.Opacity("x:Q", scale=alt.Scale(domain=[0, 1]), legend=None),
        column=alt.Column(
            "region:N",
            sort=[],
            spacing=11,
            header=alt.Header(
                labelAngle=270,
                labelPadding=-38,
                labelAlign="left",
                labelColor=colors["eco-gray"],
                labelFontSize=11,
                title="",
                labelOrient="bottom",
            ),
        ),
    )
    .transform_calculate(x="datum.itl=='UK'?0.2:1")
)
layer1 = (
    ((line).properties(height=300, width=61))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

# Fig 6

In [642]:
df = (
    pd.read_excel(
        "raw/rtisajan2022.xlsx", skiprows=6, sheet_name="11. Employees (NUTS2)"
    )
    .set_index("Date")
    .T.astype(int)
)
for c in df.columns[1:]:
    df[c] = np.round(df[c]*1000/df["July 2014"]) / 10
df["July 2014"] = 100
df1 = df.stack().reset_index()
df1.columns = ["region", "date", "value"]
df1["date"] = "28 " + df1["date"]
df1["date"] = pd.to_datetime(df1["date"])
df1["date"] = df1["date"].astype(str)
df1['NUTS']=2

df = (
    pd.read_excel(
        "raw/rtisajan2022.xlsx", skiprows=6, sheet_name="7. Employees (NUTS1)"
    )
    .set_index("Date")
    .T.astype(int)
)
for c in df.columns[1:]:
    df[c] = np.round(df[c]*1000/df["July 2014"]) / 10
df["July 2014"] = 100
df2 = df.stack().reset_index()
df2.columns = ["region", "date", "value"]
df2["date"] = "28 " + df2["date"]
df2["date"] = pd.to_datetime(df2["date"])
df2["date"] = df2["date"].astype(str)
df2['NUTS']=1

df=pd.concat([df1,df2]).drop_duplicates().reset_index().drop('index',axis=1)

In [643]:
f = "fig6_employment"
f6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,region,date,value,NUTS
0,Tees Valley and Durham,2014-07-28,100.0,2
1,Tees Valley and Durham,2014-08-28,100.2,2
2,Tees Valley and Durham,2014-09-28,100.3,2
3,Tees Valley and Durham,2014-10-28,100.4,2
4,Tees Valley and Durham,2014-11-28,100.7,2


In [644]:
base = (
    alt.Chart(f6)
    .encode(
        x=alt.X(
            "date:T",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                orient="bottom",
            ),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.region+' | '+monthFormat(month(datum.date))+' '+year(datum.date)+' | '+datum.value"
    )
)
line1 = (
    base.mark_line(strokeWidth=1.5,opacity=0.2)
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                gridOpacity=0.4,
                title="Number of people employed referenced to July 2014 = 100",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                format="s",
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=[85,120])
        ),
        color=alt.Color('region:N',scale=alt.Scale(range=[colors['eco-turquiose']]),legend=None)
    )
    .transform_filter("datum.NUTS==2")
    .transform_filter("datum.region!='UK'")
    .transform_filter("datum.region!='North Eastern Scotland'")
)
points1=line1.mark_point(size=5,opacity=0.2).transform_filter('month(datum.date)==0')
line2 = (
    base.mark_line(strokeWidth=1.5,opacity=0.7)
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                gridOpacity=0.4,
                title="Number of people employed referenced to July 2014 = 100",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                format="s",
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=[85,120])
        ),
        color=alt.Color('region:N',scale=alt.Scale(range=[colors['eco-turquiose']]),legend=None)
    )
    .transform_filter("datum.NUTS==1")
    .transform_filter("datum.region!='UK'")
    .transform_filter("datum.region!='Scotland'")
)
line3 = (
    base.mark_line(strokeWidth=3,color=colors['eco-mid-blue'])
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                gridOpacity=0.4,
                title="Number of people employed referenced to July 2014 = 100",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                format="s",
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=[85,120])
        ),
    )
    .transform_filter("datum.NUTS==1")
    .transform_filter("datum.region=='UK'")
)
line4 = (
    base.mark_line(strokeWidth=3,color=colors['eco-light-blue'])
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                gridOpacity=0.4,
                title="Number of people employed referenced to July 2014 = 100",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                format="s",
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=[85,120])
        ),
    )
    .transform_filter("datum.NUTS==1")
    .transform_filter("datum.region=='Scotland'")
)
line5 = (
    base.mark_line(strokeWidth=3,color=colors['eco-dot'])
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                gridOpacity=0.4,
                title="Number of people employed referenced to July 2014 = 100",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                format="s",
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=[85,120])
        ),
    )
    .transform_filter("datum.region=='North Eastern Scotland'")
)
label5=line5.mark_text(color=colors['eco-dot'],align='left',dx=5).encode(text='region:N').transform_filter("datum.date==toDate('2021-12-28')")
label4=line4.mark_text(color=colors['eco-light-blue'],align='left',dx=5).encode(text='region:N').transform_filter("datum.date==toDate('2021-12-28')")
label3=line3.mark_text(color=colors['eco-mid-blue'],align='left',dx=5).encode(text='region:N').transform_filter("datum.date==toDate('2021-12-28')")
layer1 = (
    ((line1+line2+line3+line4+line5+label3+label4+label5).properties(height=300, width=350))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 7

In [645]:
#copy from scottish-independence-what-are-the-big-economic-questions/visualisation/fig5/

In [646]:
f7='fig7_population_change'
readme = (
    "### "
    + f7
    + "\n[Interactive map](https://economicsobservatory.github.io/ECOvisualisations/articles/"
    + uid
    + "/visualisation/fig7/)\n\n"
)
open("README.md", "a").write(readme)

178